In [ ]:
# Install necessary library
%pip install -U geemap

In [ ]:
# Import necessary libraries
import ee
import geemap
import datetime

In [ ]:
# To initialise the project
ee.Authenticate()
ee.Initialize(project='ee-rice-yield-prediction')

# Create an interactive map
Map = geemap.Map(basemap='SATELLITE')

In [ ]:
# Load the GAUL dataset (Second-Level Administrative Units)
gaul = ee.FeatureCollection("FAO/GAUL/2015/level2")

# Filter Sabak Bernam administrative unit
sabak = gaul.filter(ee.Filter.eq("ADM2_NAME", "Sabak Bernam"))

# Draw the boundary of Sabak Bernam
sabak_boundary = ee.Image().byte().paint(
  featureCollection = sabak,
  width=3
)

# Specify the visualisation of the boundary of Sabak Bernam
sabak_vis = {
    "palette": "black"
}

# Add Sabak Bernam's boundary to the map as a layer
Map.addLayer(sabak_boundary, sabak_vis, "Sabak Bernam")

# Center the map on Sabak Bernam
Map.centerObject(sabak, 8)

# Display the map
Map

Map(center=[3.679628002569312, 101.09087962782114], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Load the NO2 images in Sabak Bernam area
no2 = (ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2')
         .select('NO2_column_number_density')
         .filterDate('2023-03-05', '2023-05-25')
         .filterBounds(sabak))

# Define the visualisation parameters
no2_vis = {
  'min': 0,
  'max': 0.0002,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}

image = no2.median()

# Adding the image to the map as a layer
Map.addLayer(image.clip(sabak), no2_vis, 'N02')

# Display map
Map

Map(bottom=32398.0, center=[3.679628002569312, 101.09087962782114], controls=(WidgetControl(options=['position…

In [ ]:
#bbox = m.user_roi_bounds()
# Sekinchan area
bbox=[101.0821, 3.5211, 101.1456, 3.5777]
print("Sekinchan Area:", bbox)

Sekinchan Area: [101.0821, 3.5211, 101.1456, 3.5777]


In [ ]:
# Define a function to calculate the mean NO2 value for each image in the collection within the specified polygon
def calculate_mean_no2(image):
    mean = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]),
        scale=1000,
        maxPixels=1e9
    )

    # Create a feature with null geometry
    # Set NO2 value and date as properties
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    feature = ee.Feature(None, {'date': date, 'no2': mean.get('NO2_column_number_density')})
    return feature

# Define start and end dates for the first quarter
start_date = '2019-03-05'
end_date = '2019-05-25'

# Initialise values
year = 1
quarter = 1

# Define the start and end dates for the second quarter
second_quarter_start_date = '2019-09-05'
second_quarter_end_date = '2019-11-25'

while (quarter <= 2):
    while (year <= 3):
        # Load the NO2 dataset
        collection = (ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2')
          .select('NO2_column_number_density')
          .filterDate(start_date, end_date)
          .filterBounds(sabak)
          .filterBounds(ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3])))

        # Map the function over the collection to calculate the mean NO2 value for each image within the specified polygon
        time_series = collection.map(calculate_mean_no2)

         # Define a name for the exported CSV file
        if (quarter==1):
          filename = "NO2_" + start_date[0:4] + "_Q1"
        elif (quarter==2):
          filename = "NO2_" + start_date[0:4] + "_Q2"

        # Export the features to a CSV file
        task = ee.batch.Export.table.toDrive(
            collection=time_series,
            description='no2_density',
            folder='no2',
            fileNamePrefix= filename,
            fileFormat='CSV'
        )
        task.start()

        # Convert the start and end dates to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

        # Increment the year by 1
        start_date_dt = start_date_dt.replace(year=start_date_dt.year + 1)
        end_date_dt = end_date_dt.replace(year=end_date_dt.year + 1)

        # Convert the datetime objects back to strings
        start_date = start_date_dt.strftime('%Y-%m-%d')
        end_date = end_date_dt.strftime('%Y-%m-%d')

        year = year + 1

    quarter = quarter + 1

    # Set the start and end dates for the second quarter
    start_date = second_quarter_start_date
    end_date = second_quarter_end_date
    year = 1

In [ ]:
# Check task status
task.status()